In [4]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

def crawl_url(url):
    # 단일 URL 크롤링: requests로 가져와 newspaper 파싱 (403시 /amp 폴백)
    try:
        r = requests.get(url, headers=HEADERS, timeout=12)
        if r.status_code == 403:
            r = requests.get(url.rstrip("/") + "/amp", headers=HEADERS, timeout=12)
        if r.status_code in (403, 404):
            return None
        r.raise_for_status()

        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()
        if not art.title or not art.text:
            return None
        return {"url": url, "title": art.title, "text": art.text, "publish_date": art.publish_date}
    except:
        return None

def crawl_g_articles(date_str, max_workers=31):
    """g 기사 크롤링 (0000~9999)"""
    urls = [f"{BASE_URL}{date_str}{num:04}g" for num in range(10000)]
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(crawl_url, u) for u in urls]
        collected = 0
        with tqdm(total=len(futures), desc=f"g {date_str}") as pbar:
            for fut in as_completed(futures):
                res = fut.result()
                if res:
                    results.append(res)
                    collected += 1
                pbar.set_postfix_str(f"수집 {collected}") # 현재 수집 건수
                pbar.update(1)

                time.sleep(0.06)  # 너무 빠른 요청 방지(차단 완화)

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    tqdm.write(f"최종 수집 기사 수: {len(df)}") # 최종 수집 건 수
    return df

In [5]:
day_str = "20250822"

In [6]:
out_df = crawl_g_articles(day_str)
out_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_g_{day_str}.csv", index=False)

out_df

g 20250822: 100%|██████████| 10000/10000 [10:51<00:00, 15.35it/s, 수집 8]


최종 수집 기사 수: 8


,url,title,text,publish_date
0,https://www.hankyung.com/article/202508224832g,"""명품백 대신 살래요"" 돌변하더니 '불티'…불황에도 '대박'","""나를 위한 작은 사치""\n\n불황에도 불티나게 팔렸다\n\n사진=신세계인터내셔날\...",2025-08-22 08:46:20+09:00
1,https://www.hankyung.com/article/202508224872g,"""가려운 곳 박박 긁어줬다""…韓 30대 푹 빠진 中 제품 정체",'중국산 필수템'…30대에 인기 폭발\n\n\n\n드리미 신제품 관심고객 2만명 넘...,2025-08-22 09:54:49+09:00
2,https://www.hankyung.com/article/202508224895g,이정후·오타니 경기 직관하러 여행 간다…'컨셉투어' 출시,로스앤젤레스 다저스타디움. 사진=모두투어\n\n메이저리그(MLB) 야구팬을 겨냥한 ...,2025-08-22 09:13:40+09:00
3,https://www.hankyung.com/article/202508225007g,빵지순례 성지 '대전'…쇼핑부터 미식까지 올인원 '몰케이션' 여행지로 주목,대전 갤러리아 타임월드. 사진=아고다(한화 갤러리아 제공)\n\n쇼핑과 엔터테인먼트...,2025-08-22 09:38:14+09:00
4,https://www.hankyung.com/article/202508225293g,"""주4일제보다 더 좋아요""…Z세대가 꼽은 '최고 복지' 뭐길래","Z세대 '입사 고민' 물었더니\n\n\n\n""주4일제보다 성과급이 좋아요""\n\n진...",2025-08-22 11:00:12+09:00
5,https://www.hankyung.com/article/202508225292g,美서 웃돈까지 붙더니 '최초 기록'…제네시스 'GV70' 일냈다 [모빌리티톡],4년 6개월 만에 누적 30만대 돌파\n\n미국서 웃돈 붙어 팔리기도\n\n내년 하...,2025-08-22 11:16:05+09:00
6,https://www.hankyung.com/article/202508225351g,"""경쟁 넘어 협력""…현대차·기아, K배터리 3사와 안전 기술 확보",LG에너지솔루션·삼성SDI·SK온과 협업 추진\n\n전기차 배터리 안전 기술 개발 ...,2025-08-22 10:50:49+09:00
7,https://www.hankyung.com/article/202508225603g,"법원, 티몬 회생절차 1년여 만에 종결 결정…""회생채권 96.5% 변제 완료""",서울 강남구 티몬 사옥의 모습. 사진=연합뉴스\n\n대규모 미정산 사태로 기업회생절...,2025-08-22 12:44:52+09:00
